In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import os
import csv

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# File to Load (Remember to Change These if my path is different)
intakes2_data_load = "Data/Austin_Animal_Center_Intakes2.csv"
outcomes2_data_load = "Data/Austin_Animal_Center_Outcomes2.csv"

intakes2_data = pd.read_csv(intakes2_data_load)
outcomes2_data = pd.read_csv(outcomes2_data_load)

In [ ]:
outcomes2_data["Animal ID"].astype(str)
intakes2_data["Animal ID"].astype(str)

In [ ]:
outcomes2_data["Animal ID"] = outcomes2_data["Animal ID"].map(lambda x: str(x)[1:])
intakes2_data["Animal ID"] = intakes2_data["Animal ID"].map(lambda x: str(x)[1:])

In [ ]:
outcomes2_data = outcomes2_data.drop(columns=['Name', 'MonthYear', 'Animal Type','Breed','Color'])
intakes2_data = intakes2_data.drop(columns=['MonthYear'])

In [ ]:
outcomes2_data = outcomes2_data.sort_values(by=["Animal ID", "DateTime"], ascending=False)
intakes2_data = intakes2_data.sort_values(by=["Animal ID", "DateTime"], ascending=False)

In [ ]:
outcomes2_data = outcomes2_data.drop_duplicates(["Animal ID"], keep= 'first')
intakes2_data = intakes2_data.drop_duplicates(["Animal ID"], keep= 'first')

In [ ]:
outcomes2_data.rename(columns = {'DateTime':'DateTime_out'}, inplace = True)

In [ ]:
# Combine the data into a single dataset.  
allanimal2_data = pd.merge(intakes2_data,outcomes2_data, how='left', on=["Animal ID","Animal ID"])
allanimal2_data

In [ ]:
allanimal2_data["DateTime"]= pd.to_datetime(allanimal2_data["DateTime"])

In [ ]:
allanimal2_data["DateTime_out"]= pd.to_datetime(allanimal2_data["DateTime_out"])

In [ ]:
allanimal2_data["shelter_time"] = allanimal2_data["DateTime_out"] - allanimal2_data["DateTime"]

In [ ]:
allanimal2_data.loc[allanimal2_data['shelter_time'] > pd.Timedelta(0)]

In [ ]:
allanimal2_data["shelter_time"] = allanimal2_data["shelter_time"].astype(str)

In [ ]:
allanimal2_data.dtypes

In [ ]:
allanimal2_data["shelter_days"] = allanimal2_data["shelter_time"].str[:2]

In [ ]:
allanimal2_data

In [ ]:
allanimal2_data["shelter_days"] = allanimal2_data["shelter_days"].replace("Na", 0.1)

In [ ]:
allanimal2_data

In [ ]:
allanimal2_data["shelter_days"] = allanimal2_data["shelter_days"].astype(int)

In [ ]:
allanimal2_data.loc[allanimal2_data["shelter_days"].isin(range(0,1)), "Adopt Rating"] = "AA"
allanimal2_data.loc[allanimal2_data["shelter_days"].isin(range(2,5)), "Adopt Rating"] = "A"
allanimal2_data.loc[allanimal2_data["shelter_days"].isin(range(6,14)), "Adopt Rating"] = "B"
allanimal2_data.loc[allanimal2_data["shelter_days"].isin(range(15,30)), "Adopt Rating"] = "C"
allanimal2_data.loc[allanimal2_data["shelter_days"].isin(range(31,90)), "Adopt Rating"] = "D"
allanimal2_data.loc[allanimal2_data["shelter_days"].isin(range(91,1000)), "Adopt Rating"] = "F"


In [ ]:
allanimal2_data

In [ ]:
allanimal2_data.loc[allanimal2_data["Outcome Type"]!="Adoption", "Adopt Rating"] = "Not Adopted"

In [ ]:
allanimal2_data

In [ ]:
allanimal2_data.loc[allanimal2_data["shelter_time"]=="NaT", "shelter_days"] = "New intake"

In [ ]:
allanimal2_data

In [ ]:
allanimal2_data.to_csv("all_animals_merge.csv", index=False)

In [ ]:
adoption_speed = allanimal2_data.groupby("Adopt Rating")

In [ ]:
speed_data = adoption_speed["Adopt Rating"].count()

ax1 = speed_data.plot.bar(facecolor='magenta',title="Adoption Ratings")
ax1.set_xlabel("Rating")
ax1.set_ylabel("Count")

In [ ]:
allanimal2_data["Outcome Type"] = allanimal2_data["Outcome Type"].replace("NaN", -1)

In [ ]:
outcome = allanimal2_data.groupby("Outcome Type")

In [ ]:
outcome_data = outcome["Outcome Type"].count()

ax1 = outcome_data.plot.bar(facecolor='purple')
ax1.set_xlabel("Outcome")
ax1.set_ylabel("Count")

In [ ]:
sex = allanimal2_data.groupby("Sex upon Outcome")
sex_data = sex["Sex upon Outcome"].count()

ax1 = sex_data.plot.bar(facecolor='blue')
ax1.set_xlabel("Male/Female")
ax1.set_ylabel("Count")

In [ ]:
import seaborn as sns

In [ ]:
sns.catplot(x = "x",       # x variable name
            y = "y",       # y variable name
            hue = "type",  # group variable name
            data = df,     # dataframe to plot
            kind = "bar")

In [ ]:
species = allanimal2_data.groupby("Animal Type")
species_data = species["Animal Type"].count()

ax1 = allanimal2_data.plot(kind='bar', x='species', y='sex', rot=90, facecolor='orange')
ax1.set_xlabel("Species")
ax1.set_ylabel("Male/Female")

In [ ]:
#adopted2_animals = allanimal2_data[allanimal2_data["Outcome Type"]=="Adoption"]
#adopted2_animals.head()

In [ ]:
#animal_data = adopted_animals.drop_duplicates(["Animal ID"], keep= 'first')

In [ ]:
type_cat = animal_data.loc[animal_data["Animal Type"] == "Cat"]
type_dog = animal_data.loc[animal_data["Animal Type"] == "Dog"]
type_bird = animal_data.loc[animal_data["Animal Type"] == "Bird"]
type_livestock = animal_data.loc[animal_data["Animal Type"] == "Livestock"]
type_other = animal_data.loc[animal_data["Animal Type"] == "Other"]

In [ ]:
    type_cats = type_cat[type_cat['shelter_time'] > pd.Timedelta(0)]
    type_dogs = type_dog[type_dog['shelter_time'] > pd.Timedelta(0)]
    type_birds = type_bird[type_bird['shelter_time'] > pd.Timedelta(0)]
    type_livestocks = type_livestock[type_livestock['shelter_time'] > pd.Timedelta(0)]
    type_others = type_other[type_other['shelter_time'] > pd.Timedelta(0)]

In [ ]:
type_cats.to_csv("adopted_cats_clean.csv", index=False)
type_dogs.to_csv("adopted_dogs_clean.csv", index=False)
type_birds.to_csv("adopted_birds_clean.csv", index=False)
type_livestocks.to_csv("adopted_livestock_clean.csv", index=False)
type_others.to_csv("adopted_other_clean.csv", index=False)